<a href="https://colab.research.google.com/github/hauntedcupoftea/nikolaj/blob/main/Optimal_Routes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FINDING OPTIMAL ROUTES

In [ ]:
# !pip install osmnx
# !pip install networkx

In [ ]:
import osmnx as ox
import networkx as nx

In [ ]:
import numpy as np
import pandas as pd
import csv
import datetime as dt
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/hauntedcupoftea/nikolaj/main/datasets/crimedata2016.csv')
df.head()

In [5]:
timelist = []
for i in range(len(df)):
    datetime_object = dt.datetime.strptime(df['Date'][i][-11:], '%I:%M:%S %p')
    timelist.append(datetime_object)
df['Time'] = timelist
df.head()

NameError: name 'df' is not defined

In [ ]:
cleandf = df.drop(['Date', 'X Coordinate', 'Y Coordinate', 'Beat', 'Year', 'FBI Code'], axis=1)
cleandf.head()

In [ ]:
# using sklearn's label encoder for more complex encoders where severity isn't needed
from sklearn import preprocessing as pp
lp = pp.LabelEncoder()
op = pp.OrdinalEncoder()
# creating a manual encoder for descriptions
basicCrime = list(set(cleandf['Primary Type']))
basicCrime

In [ ]:
# manual encoding of crimes based on severity
# in order to generate a heatmap of which areas are the most dangerous
primList = {'NON - CRIMINAL': 0, 'NON-CRIMINAL (SUBJECT SPECIFIED)': 0, 'NON-CRIMINAL': 0,
            'INTIMIDATION': 1, 'OBSCENITY': 1, 'OTHER OFFENSE': 1, 'PUBLIC INDECENCY': 1,
            'LIQUOR LAW VIOLATION': 2, 'PUBLIC PEACE VIOLATION': 2, 'CONCEALED CARRY LICENSE VIOLATION': 2,
            'PROSTITUTION': 3, 'GAMBLING': 3, 'INTERFERENCE WITH PUBLIC OFFICER': 3, 'STALKING': 3,
            'ARSON': 6, 'BURGLARY': 5, 'BATTERY': 2, 'ROBBERY': 5, 'SEX OFFENSE': 5, 'ASSAULT': 3,
            'THEFT': 4, 'DECEPTIVE PRACTICE': 5, 'CRIMINAL TRESPASS': 4, 'CRIMINAL DAMAGE': 4, 'WEAPONS VIOLATION' : 5,
            'MOTOR VEHICLE THEFT': 5, 'OFFENSE INVOLVING CHILDREN': 5, 'KIDNAPPING': 5, 'NARCOTICS': 5,
            'OTHER NARCOTIC VIOLATION' : 4,'HUMAN TRAFFICKING' : 6,'CRIM SEXUAL ASSAULT' : 6, 'HOMICIDE' : 6}
len(primList)

In [ ]:
# running the encode, along with the other encoders and adding them all to the dataframe
encodePrim = [primList[i] for i in cleandf['Primary Type']]
cleandf['desc'] = lp.fit_transform(cleandf['Description'])
cleandf['locdesc'] = lp.fit_transform(cleandf['Location Description'])
cleandf['type'] = encodePrim
cleandf.head()

In [ ]:
from collections import Counter
count = Counter(cleandf['Location'])
print(count)

In [ ]:
# colormap = {0 : '#0d0887', 1 : '#5d02a4', 2 : '#9b169e', 3 : '#c94579', 
#             4 : '#ed7953', 5 : '#fcb331', 6 : '#f0f821'}
# plt.scatter(cleandf['Longitude'], cleandf['Latitude'], marker='.', 
#             c=[colormap[i] for i in cleandf['type']])
# plt.show()

In [ ]:
# making a time filter
def timeFilter(start: str, end: str) -> pd.DataFrame:
    start = dt.datetime.strptime(start, '%H:%M:%S')
    end = dt.datetime.strptime(end, '%H:%M:%S')
    if (start < end):
        return cleandf.loc[(df['Time'] >= start) & (df['Time'] < end)]
    else:
        return cleandf.loc[(df['Time'] >= start) | (df['Time'] < end)]

# examplse of if
ifs = timeFilter('17:00:00', '09:00:00')
# example of else
els = timeFilter('09:00:00','17:00:00')
# check to see if we cover everything
print(len(ifs) + len(els) == len(cleandf))
els.shape

In [ ]:
cleandf.head()

In [ ]:
from sklearn.cluster import *
def one():
    nCluster = int(input())
#    color = list(np.random.choice(range(256), size=nCluster))
    model = KMeans(n_clusters=nCluster)
    results = model.fit_predict(cleandf.loc(axis=1)['Latitude':'Longitude'])
#    plt.scatter(cleandf['Longitude'], cleandf['Latitude'], marker='.', 
#                c=[color[i] for i in results])
    return results

In [ ]:
results = one()
color = list(np.random.choice(range(256), size=len(set(results))))
cleandf['cluster'] = results

In [ ]:
from scipy.spatial import Delaunay
def alpha_shape(points, alpha, only_outer=True):
    assert points.shape[0] > 3, "Need at least four points"
    def add_edge(edges, i, j):
        if (i, j) in edges or (j, i) in edges:
            assert (j, i) in edges, "Can't go twice over same directed edge right?"
            if only_outer:
                edges.remove((j, i))
            return
        edges.add((i, j))
    tri = Delaunay(points)
    edges = set()
    for ia, ib, ic in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        a = np.sqrt((pa[0] - pb[0]) ** 2 + (pa[1] - pb[1]) ** 2)
        b = np.sqrt((pb[0] - pc[0]) ** 2 + (pb[1] - pc[1]) ** 2)
        c = np.sqrt((pc[0] - pa[0]) ** 2 + (pc[1] - pa[1]) ** 2)
        s = (a + b + c) / 2.0
        area = np.sqrt(s * (s - a) * (s - b) * (s - c))
        circum_r = a * b * c / (4.0 * area)
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, ia)
    return edges

In [ ]:
import warnings
warnings.filterwarnings("ignore")
# Plotting the output
fig, ax = plt.subplots(figsize=(30, 8))
for i in range(len(set(results))):
    fildf = cleandf[cleandf['cluster'] == i]
    nmod = KMeans(int(np.power(len(fildf), 0.25)))
    nmod.fit([[i, j] for i, j in zip(fildf['Longitude'], fildf['Latitude'])])
    centers = nmod.cluster_centers_
    points = np.array([[i, j] for i, j in zip(fildf['Longitude'], fildf['Latitude'])])
    try:
        edges = alpha_shape(centers, alpha=4, only_outer=True)
    except:
        continue
    plt.subplot(1, 3, 1)
    plt.plot(points[:, 0], points[:, 1], '.')
    # USE EDGES AND CENTERS TO PLOT NORMAL ROUTES, REMOVE POINTS AND ALL REFERENCES IN FINAL CODE
    for i, j in edges:
        plt.plot(centers[[i, j], 0], centers[[i, j], 1])
plt.show()

In [ ]:
print(edges)

In [ ]:
print(centers)

In [ ]:
centers[1]

In [ ]:
centers[0]

In [ ]:
# ox.config(log_console=True, use_cache=True)
# # define the start and end locations in latlng
# # start_latlng = (37.78497,-122.43327)
# # end_latlng = (37.78071,-122.41445)

# start_latlang = centers[0]
# end_latlng = centers[2]
# # location where you want to find your route
# place     = 'Chicago, Illinois, United States'
# # find shortest route based on the mode of travel
# mode      = 'drive'        # 'drive', 'bike', 'walk'
# # find shortest path based on distance or time
# optimizer = 'time'        # 'length','time'
# # create graph from OSM within the boundaries of some 
# # geocodable place(s)
# graph = ox.graph_from_place(place, network_type = mode)
# # find the nearest node to the start location
# orig_node = ox.get_nearest_node(graph, start_latlng)
# # find the nearest node to the end location
# dest_node = ox.get_nearest_node(graph, end_latlng)
# #  find the shortest path
# shortest_route = nx.shortest_path(graph,
#                                   orig_node,
#                                   dest_node,
#                                   weight=optimizer)

In [ ]:

# shortest_route_map = ox.plot_route_folium(graph, shortest_route)
# shortest_route_map

In [ ]:
def find_route(st,end):
  ox.config(log_console=True, use_cache=True)
  # define the start and end locations in latlng
  # start_latlng = (37.78497,-122.43327)
  # end_latlng = (37.78071,-122.41445)

  start_latlang = st
  end_latlng = end
  # location where you want to find your route
  place     = 'Chicago, Illinois, United States'
  # find shortest route based on the mode of travel
  mode      = 'drive'        # 'drive', 'bike', 'walk'
  # find shortest path based on distance or time
  optimizer = 'time'        # 'length','time'
  # create graph from OSM within the boundaries of some 
  # geocodable place(s)
  graph = ox.graph_from_place(place, network_type = mode)
  # find the nearest node to the start location
  orig_node = ox.get_nearest_node(graph, start_latlang)
  # find the nearest node to the end location
  dest_node = ox.get_nearest_node(graph, end_latlng)
  #  find the shortest path
  shortest_route = nx.shortest_path(graph,
                                    orig_node,
                                    dest_node,
                                    weight=optimizer)
 
  shortest_route_map = ox.plot_route_folium(graph, shortest_route)
  return shortest_route_map
find_route(centers[0],centers[1])

In [ ]:
# import gmplot  
# latitudeList = [ 28.691234, 28.818390, 29.089301 ]  
# longitudeList = [ 77.193802, 77.023890, 76.865211 ]  
# myGmap = gmplot.GoogleMapPlotter(28.612894, 77.229446, 11)  
# myGmap.scatter( latitudeList, longitudeList, '#FF0000', size = 40, marker = False )  
# # drawing a polygon using the polygon() method  
# # of the GoogleMapPlotter class with the help of coordinates  
# myGmap.polygon( latitudeList, longitudeList, color = 'cornflowerblue' )  
